## Convert Object Detection Label into Image Classification

In [5]:
import os
from PIL import Image

error_list = []

# Set the base paths to the images and annotations folders
base_images_path = './yolov8-dataset/'
base_annotations_path = './yolov8-dataset/'

# Set the base path to the output folder
base_output_path = './yolov8-dataset-extracted/'

# Set the dataset splits
splits = ['train', 'valid', 'test']

# Loop through the dataset splits
for split in splits:
    # Set the paths to the images and annotations folders for the current split
    images_path = os.path.join(base_images_path, split, 'images')
    annotations_path = os.path.join(base_annotations_path, split, 'labels')

    # Set the path to the output folder for the current split
    output_path = os.path.join(base_output_path, split)

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    error = []

    # Loop through the annotation files
    for filename in os.listdir(annotations_path):
        # Open the annotation file
        with open(os.path.join(annotations_path, filename), 'r') as f:
            # Read the annotation data
            data = f.read().strip().split('\n')
            
            # Loop through the objects in the image
            for obj in data:
                # Split the object data
                obj_data = obj.split(' ')
                
                try:
                    # Get the class label and bounding box coordinates
                    class_label = obj_data[0]
                    x_center, y_center, width, height = map(float, obj_data[1:])
                except ValueError:
                    # print(f'Error processing file: {filename}')
                    # print(f'Object data: {obj_data}')
                    error.append(filename)
                    # continue
                    break
                
                # Open the image file
                image_filename = filename.replace('.txt', '.jpg')
                image = Image.open(os.path.join(images_path, image_filename))
                
                # Calculate the bounding box coordinates
                image_width, image_height = image.size
                x1 = int((x_center - width / 2) * image_width)
                y1 = int((y_center - height / 2) * image_height)
                x2 = int((x_center + width / 2) * image_width)
                y2 = int((y_center + height / 2) * image_height)
                
                # Crop the image
                cropped_image = image.crop((x1, y1, x2, y2))
                
                # Create the class label folder if it doesn't exist
                class_folder = os.path.join(output_path, class_label)
                if not os.path.exists(class_folder):
                    os.makedirs(class_folder)
                
                # Save the cropped image
                cropped_image.save(os.path.join(class_folder, f'{os.path.splitext(image_filename)[0]}_{class_label}.jpg'))
    print(f'total error in {split}: {len(error)}')
    error_list.append(error)



total error in train: 1395
total error in valid: 26
total error in test: 26


In [7]:
error_list[0]

1395